## Git clone

Cloning this repository after you download this ipynb file.

In [1]:
import getpass, os

# 这里让你在运行时输入 token, 不会显示在输出里
#token = getpass.getpass("input Token: ")
username = "qiulinfan"
owner = "qiulinfan" 
repo = "graphNL2SQL"

#os.environ["GITHUB_TOKEN"] = token
#os.environ["GITHUB_USER"] = username
os.environ["GITHUB_OWNER"] = owner
os.environ["GITHUB_REPO"] = repo

!git clone https://github.com/${GITHUB_OWNER}/${GITHUB_REPO}.git
%cd graphNL2SQL/

🐟 Welcome back, QL! Hail to the Elden Lord! 
Running in WSL: Ubuntu-22.04
fish: Variables cannot be bracketed. In fish, please use {$GITHUB_REPO}.
git clone https://github.com/${GITHUB_OWNER}/${GITHUB_REPO}.git
                                              ^
[Errno 2] No such file or directory: 'graphNL2SQL/'
/home/qiulin/Documents/graphNL2SQL


In [2]:
!git pull

🐟 Welcome back, QL! Hail to the Elden Lord! 
Running in WSL: Ubuntu-22.04
Already up to date.


## Download and preprocess data

In [ ]:
!python scripts/download_data.py

🐟 Welcome back, QL! Hail to the Elden Lord! 
Running in WSL: Ubuntu-22.04
NL2SQL Dataset Downloader

[1/2] WikiSQL Dataset
----------------------------------------
WikiSQL: 100%|█████████████████████████████| 26.2M/26.2M [00:06<00:00, 4.17MB/s]
Extracting WikiSQL...
/home/qiulin/Documents/graphNL2SQL/download_data.py:52: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(DATA_DIR)
WikiSQL extracted to /home/qiulin/Documents/graphNL2SQL/data/wikisql

[2/2] Spider Dataset
----------------------------------------
Installing gdown...
Downloading...
From (original): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J
From (redirected): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J&confirm=t&uuid=d9a8635d-60e9-4736-84a0-3a0504ea5b56
To: /home/qiulin/Documents/graphNL2SQL/data/spider_data.zip
100%|███████████████████████████

In [ ]:
!python scripts/prepare_training_data.py --semantic --semantic-threshold 0.8 --wikisql-balanced 5000 --spider

🐟 Welcome back, QL! Hail to the Elden Lord! 
Running in WSL: Ubuntu-22.04
Processing WikiSQL train split...
Sampling 5000 balanced WikiSQL examples from train...
  Pattern distribution in dataset:
    where_only: 40598
    count: 5114
    min: 3231
    max: 3161
    avg: 2201
    sum: 2042
    select_only: 8
  Sampled 4293 examples
WikiSQL: 100%|███████████████████████████| 4293/4293 [00:00<00:00, 65487.38it/s]
Processed 4293 WikiSQL examples
Processing WikiSQL dev split...
Sampling 1000 balanced WikiSQL examples from dev...
  Pattern distribution in dataset:
    where_only: 6017
    count: 779
    max: 507
    min: 468
    avg: 329
    sum: 321
    select_only: 0
  Sampled 999 examples
WikiSQL: 100%|█████████████████████████████| 999/999 [00:00<00:00, 64631.27it/s]
Processed 999 WikiSQL examples
Saved 4293 examples to /home/qiulin/Documents/graphNL2SQL/training_data/wikisql_train.jsonl
Saved 999 examples to /home/qiulin/Documents/graphNL2SQL/training_data/wikisql_dev.jsonl
Processing 

In [5]:
import pandas as pd
pd.set_option("display.max_rows", None)   # 不省略行
pd.set_option("display.max_columns", None)  # 不省略列
pd.set_option("display.max_colwidth", None) # 不省略内容长度
df = pd.read_json("training_data/spider_train.jsonl", lines=True)
df['schema'][0:10]

0    [DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n    Department_ID (PK)\n    Name\n    Creation\n    Ranking\n    Budget_in_Billions\n    Num_Employees\nhead:\n    head_ID (PK)\n    name\n    born_state\n    age\nmanagement:\n    department_ID (PK, FK)\n    head_ID (FK)\n    temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID
1    [DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n    Department_ID (PK)\n    Name\n    Creation\n    Ranking\n    Budget_in_Billions\n    Num_Employees\nhead:\n    head_ID (PK)\n    name\n    born_state\n    age\nmanagement:\n    department_ID (PK, FK)\n    head_ID (FK)\n    temporary_acting\n\n[FOREIGN KEYS]\nmanagement.head_ID -> head.head_ID\nmanagement.department_ID -> department.Department_ID
2    [DATABASE]\ndepartment_management\n\n[TABLES]\ndepartment:\n    Department_ID (PK)\n    Name\n    Creation\n    Ranking\n    Budget_in_Billions\n    Num_Employee

In [6]:
import pandas as pd
df = pd.read_json("training_data/wikisql_train.jsonl", lines=True)
df['schema'][0:10]

0                                                                                                                                     [TABLES]\nManila:\n    District (PK)\n    s Barangay\n    Population (2010 census)\n    Area ( has .)\n    Pop. density (per km2)
1    [TABLES]\nList_of_rural_municipalities_in_Saskatchewan:\n    Rural municipality (RM) (PK)\n    RM No.\n    SARM Div. No.\n    Census Div. No.\n    Population (2011)\n    Population (2006)\n    Change (%)\n    Land area (km²)\n    Population density (per km²)
2                                                                                                                 [TABLES]\nKangxi_radical:\n    Radical (variants) (PK)\n    Stroke count\n    Pīnyīn\n    Hiragana - Romaji\n    Meaning\n    Frequency\n    Examples
3                                                                                                                                       [TABLES]\nSouth_West_District_Football_League:\n    South West DFL (PK)\

## Training

### Install Dependencies

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes torch wandb --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 38.2 MB/s eta 0:00:00


In [ ]:
!pip install -r requirements_train.txt

In [67]:
import transformers
import peft
import accelerate

print(f"transformers: {transformers.__version__}")
print(f"peft: {peft.__version__}")
print(f"accelerate: {accelerate.__version__}")

transformers: 4.57.2
peft: 0.18.0
accelerate: 1.12.0


In [78]:
import os
import json
from pathlib import Path
from datetime import datetime

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
)

import wandb

### GPU Check

In [ ]:
import torch

print("=" * 60)
print("GPU INFORMATION")
print("=" * 60)

if torch.cuda.is_available():
    print(f"✓ CUDA available: True")
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    total_mem = props.total_memory / 1e9
    print(f"  Memory: {total_mem:.2f} GB")
    print(f"  CUDA Version: {torch.version.cuda}")

    # Recommend config based on VRAM
    if total_mem < 12:
        print(f"\n  Recommended config: small_gpu (4-bit quantization)")
    elif total_mem < 24:
        print(f"\n Recommended config: default")
    else:
        print(f"\n Recommended config: large_gpu (more LoRA capacity)")
else:
    print("✗ CUDA not available - Training will be VERY slow on CPU")

print("=" * 60)

### Configuration

Define hyperparameters and LoRA configuration. (No need to run this cell if config.json already good.)

In [ ]:
import json
import sys
sys.path.insert(0, '.')  # Add project root to path

# =============================================================================
# CONFIGURATION - Edit values below, then run this cell to save
# =============================================================================

CONFIG = {
    "model": {
        "model_name": "Qwen/Qwen2.5-7B-Instruct",  # or "microsoft/Phi-3.5-mini-instruct"
        "load_in_4bit": True,   # Recommended for <16GB VRAM
        "load_in_8bit": False,
    },
    "lora": {
        "r": 16,                # Rank: 8, 16, 32, 64
        "alpha": 32,            # Typically 1-2x r
        "dropout": 0.1,
        "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj"],
    },
    "training": {
        "wikisql_epochs": 0,    # Phase 1: WikiSQL warmup
        "spider_epochs": 2,     # Phase 2: Spider main training
        "batch_size": 2,
        "gradient_accumulation": 8,  # Effective batch = batch_size * gradient_accumulation
        "learning_rate": 1e-4,
        "lr_scheduler": "cosine",
        "warmup_ratio": 0.05,
        "max_seq_length": 1024,
        "gradient_checkpointing": True,
        "use_bf16": True,
        "use_fp16": False,
    },
    "data": {
        "data_dir": "./training_data",
        "output_dir": "./checkpoints",
        "max_train_samples": None,  # None = use all
        "max_eval_samples": 500,
    },
    "wandb": {
        "enabled": True,
        "project": "nl2sql-finetuning",
        "run_name": None,  # Auto-generated if None
    },
    "save": {
        "strategy": "epoch",
        "total_limit": 5,
    },
}

# Save configuration to JSON file
with open('config.json', 'w') as f:
    json.dump(CONFIG, f, indent=2)

print("Configuration saved to config.json")
print(f"  Model: {CONFIG['model']['model_name']}")
print(f"  LoRA rank: {CONFIG['lora']['r']}, alpha: {CONFIG['lora']['alpha']}")
print(f"  Batch size: {CONFIG['training']['batch_size']} x {CONFIG['training']['gradient_accumulation']} = {CONFIG['training']['batch_size'] * CONFIG['training']['gradient_accumulation']} effective")
print(f"  Training: {CONFIG['training']['wikisql_epochs']} epoch WikiSQL + {CONFIG['training']['spider_epochs']} epochs Spider")

Configuration saved to config.json
  Model: Qwen/Qwen2.5-7B-Instruct
  LoRA rank: 16, alpha: 32
  Batch size: 2 x 8 = 16 effective
  Training: 1 epoch WikiSQL + 3 epochs Spider


### Load Datasets & Run Training

Using encapsulated functions from `training_utils.py`

In [ ]:
from scripts.training_utils import (
    TrainingConfig,
    load_config_from_json,
    save_config_to_json,
    print_config,
    load_datasets,
    show_sample_examples,
    load_model_and_tokenizer,
    setup_lora,
    print_model_parameters,
    init_wandb,
    train_phase1_wikisql,
    train_phase2_spider,
    finish_training,
)
print("Training utilities imported!")

In [ ]:
# Load configuration from JSON file
config = load_config_from_json('config.json')
print_config(config)

In [ ]:
# Load datasets
wikisql_train, wikisql_dev, spider_train, spider_dev = load_datasets(config.data_dir)
show_sample_examples(wikisql_train, spider_train)

In [ ]:
# Load model and setup LoRA
model, tokenizer = load_model_and_tokenizer(config)
model = setup_lora(model, config)
print_model_parameters(model)

In [ ]:
# Initialize WandB
run_name = init_wandb(config)

### Phase 1: WikiSQL Warmup

In [ ]:
# Phase 1: WikiSQL Warmup Training
train_phase1_wikisql(model, tokenizer, wikisql_train, wikisql_dev, config, run_name)

### Phase 2: Spider Main Training

In [ ]:
# Phase 2: Spider Main Training
trainer = train_phase2_spider(model, tokenizer, spider_train, spider_dev, config, run_name)

### Finish Training

In [ ]:
# Finish training and cleanup
finish_training(config)

In [ ]:
# Save the final model
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Model saved to {OUTPUT_DIR}")

# Finish WandB run
wandb.finish()
print("WandB run finished. Check your dashboard for loss graphs!")

Model saved to ./t5_wikisql_lora_finetuned


train/epoch,▁
train/global_step,▁
total_flos,9297442897920.0
train/epoch,3
train/global_step,3
train_loss,3.20488
train_runtime,3.245
train_samples_per_second,4.622
train_steps_per_second,0.924


WandB run finished. Check your dashboard for loss graphs!


### Download to local

In [ ]:
DOWNLOAD_PATH = "./checkpoints/phase2_spider/final"

def zip_and_download(model_path):
    model_path = Path(model_path)
    if not model_path.exists():
        print(f" Path not found: {model_path}")
        return

    zip_name = f"{model_path.name}_lora_adapter"
    zip_path = f"/tmp/{zip_name}"

    print(f" Creating zip: {model_path}")
    shutil.make_archive(zip_path, 'zip', model_path.parent, model_path.name)

    zip_file = f"{zip_path}.zip"
    print(f"   Size: {os.path.getsize(zip_file)/(1024*1024):.1f} MB")

    try:
        from google.colab import files
        print(" Downloading...")
        files.download(zip_file)
    except ImportError:
        print(f"\n Download manually: {zip_file}")

zip_and_download(DOWNLOAD_PATH)

## Testing

Using encapsulated functions from `testing_utils.py`

In [ ]:
from scripts.testing_utils import (
    generate_sql,
    load_finetuned_model,
    # Evaluation functions
    evaluate_model,              # Exact Match only
    evaluate_with_execution,     # Exact Match + Execution Match
    show_evaluation_examples,
    # Testing functions
    run_quick_test,
    run_interactive_test,
    test_with_university_schema,
    test_with_ecommerce_schema,
    UNIVERSITY_SCHEMA,
    ECOMMERCE_SCHEMA,
    SAMPLE_QUESTIONS,
    # EGD functions
    generate_sql_with_egd,
    evaluate_with_egd,
)
print("Testing utilities imported (including Execution Match and EGD)!")

### Load Fine-tuned Model

In [ ]:
# Configuration
CHECKPOINT_PATH = "./checkpoints/phase2_spider/final"

# Load fine-tuned model (or use already loaded model from training)
from pathlib import Path
if Path(CHECKPOINT_PATH).exists():
    eval_model, eval_tokenizer = load_finetuned_model(CHECKPOINT_PATH)
else:
    print(f"Checkpoint not found: {CHECKPOINT_PATH}")
    print("Using model from training session...")
    eval_model, eval_tokenizer = model, tokenizer

### Quick Test

In [ ]:
# Run quick test with default schema/question
run_quick_test(eval_model, eval_tokenizer)

### Evaluate on Spider Dev Set

In [ ]:
# Evaluate on Spider dev set
EVAL_MAX_SAMPLES = 100  # Set to None for full evaluation
USE_EGD = True          # Use Execution-Guided Decoding for better accuracy
EGD_CANDIDATES = 5      # Number of candidates for EGD

if spider_dev:
    eval_results = evaluate_model(
        eval_model, eval_tokenizer,
        spider_dev,
        max_samples=EVAL_MAX_SAMPLES,
        use_egd=USE_EGD,
        egd_candidates=EGD_CANDIDATES,
    )
    
    # Show example predictions
    show_evaluation_examples(eval_results, num_correct=3, num_incorrect=3)
else:
    print("Spider dev data not loaded.")

### Evaluate with Execution Match (EM + EX)

Execution Match compares the actual execution results of predicted SQL vs gold SQL on a mock database. This is more forgiving than Exact Match since different SQL queries can produce the same results.


In [ ]:
# Evaluate with both Exact Match (EM) and Execution Match (EX)
# This requires DuckDB: pip install duckdb
# Uses EGD setting from above cell

if spider_dev:
    ex_results = evaluate_with_execution(
        eval_model, eval_tokenizer,
        spider_dev,
        max_samples=EVAL_MAX_SAMPLES,
        verbose=True,
        use_egd=USE_EGD,
        egd_candidates=EGD_CANDIDATES,
    )
    
    print("\n" + "=" * 60)
    print("SUMMARY" + (" (with EGD)" if USE_EGD else ""))
    print("=" * 60)
    print(f"Exact Match (EM):     {ex_results['exact_match_accuracy']:.2f}%")
    print(f"Execution Match (EX): {ex_results['execution_match_accuracy']:.2f}%")
    
    # Show how many queries EX found correct that EM missed
    ex_only = sum(1 for r in ex_results['results'] 
                  if r['execution_match'] and not r['exact_match'])
    if ex_only > 0:
        print(f"\nEX found {ex_only} additional correct queries beyond EM!")
else:
    print("Spider dev data not loaded.")


### Interactive Testing

In [ ]:
# Test with university schema
test_with_university_schema(eval_model, eval_tokenizer)

In [ ]:
# Test with ecommerce schema
test_with_ecommerce_schema(eval_model, eval_tokenizer)

In [ ]:
# Custom test - modify schema and questions as needed
custom_schema = """[TABLES]
employee:
    id (PK)
    name
    department
    salary"""

custom_questions = [
    "What is the average salary?",
    "List employees in the Engineering department.",
    "Who has the highest salary?",
]

run_interactive_test(eval_model, eval_tokenizer, custom_schema, custom_questions)

interactive testing with EGD

In [ ]:
# Test EGD on a single example
test_schema = UNIVERSITY_SCHEMA
test_question = "How many students are enrolled in each course?"

egd_result = generate_sql_with_egd(
    eval_model, eval_tokenizer,
    test_question,
    test_schema,
    num_candidates=5,
    verbose=True
)

print(f"\nFinal SQL: {egd_result['sql']}")
print(f"Selection method: {egd_result['method']}")

In [ ]:
# Evaluate with EGD on Spider dev set
# EGD generates multiple candidates and selects the best executable one
EGD_EVAL_SAMPLES = 50  # Fewer samples since EGD is slower

if spider_dev:
    egd_results = evaluate_with_egd(
        eval_model, eval_tokenizer,
        spider_dev,
        max_samples=EGD_EVAL_SAMPLES,
        num_candidates=5,
        verbose=True
    )
    
    print("\n" + "=" * 60)
    print("EGD EVALUATION SUMMARY")
    print("=" * 60)
    print(f"Exact Match (EM):     {egd_results['exact_match_accuracy']:.2f}%")
    print(f"Execution Match (EX): {egd_results['execution_match_accuracy']:.2f}%")
    print(f"EGD Execution Rate:   {egd_results['execution_rate']:.2f}%")
else:
    print("Spider dev data not loaded.")

## Possible Improvement directions



1. **Input Format**: WikiSQL model expects `translate English to SQL: <question> context: <columns>`

2. **More Data**: The sample dataset is tiny. Use hundreds or thousands of examples for real training.

3. **LoRA Rank**: Start with r=16. Increase to 32 or 64 if underfitting, decrease to 8 if overfitting.

4. **Learning Rate**: 1e-4 to 3e-4 typically works well for LoRA.